In [42]:
import pandas as pd
import numpy as np
import plotly.express as px

In [18]:
df = pd.read_csv('../data/nlf_mock_drafts_23.csv')

In [19]:
num_filter = df.groupby(['player'])['mock_draft'].count().reset_index(name='num_occurrences')
avg_pos_filter = df.groupby(['player'])['pick_number'].mean().reset_index(name='adp')

In [20]:
df = pd.merge(df, num_filter, how='left', on='player')
df = pd.merge(df, avg_pos_filter, how='left', on='player')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 0 to 309
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mock_draft       310 non-null    object 
 1   authors          310 non-null    object 
 2   player           310 non-null    object 
 3   pick_number      310 non-null    int64  
 4   last_updated     10 non-null     object 
 5   url              160 non-null    object 
 6   num_occurrences  310 non-null    int64  
 7   adp              310 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 29.9+ KB


In [51]:
composite = df[df['num_occurrences']>=3].groupby(['player'])['adp'].mean().sort_values()[:31].reset_index()

In [60]:
top_first = list(composite['player'])

In [52]:
composite['pick_number'] = composite['adp'].rank().astype('int')

In [53]:
composite['mock_draft'] = 'Composite ADP'
composite['authors'] = 'Composite'
composite['last_updated'] = '04/25'
composite['url'] = np.nan
composite['num_occurrences'] = np.nan

In [56]:
df = pd.concat([composite[['mock_draft', 'authors', 'player', 'pick_number', 'last_updated', 'url', 'num_occurrences', 'adp']],
          df])

In [63]:
df = df[df.player.isin(top_first)]

In [105]:
df_heatmap = pd.pivot_table(df, 
                            index=['player', 'adp'], 
                            columns='pick_number',
                            values = 'mock_draft', 
                            aggfunc='count',
                            fill_value=0).reset_index()

In [106]:
df_heatmap = df_heatmap.sort_values(by='adp')

In [109]:
df_heatmap = df_heatmap.drop(columns='adp')

In [110]:
df_heatmap = df_heatmap.set_index('player')

In [116]:
fig = px.imshow(df_heatmap,
                color_continuous_scale=['#FFFFFF', '#0085CA'])

fig.update_layout(autosize=False, 
                  showlegend=False,
                  width=900,
                  height=800)

fig.update_traces(showlegend=False)

fig.show()

In [121]:
df_heatmap.var()

pick_number
1     3.236559
2     0.903226
3     1.169892
4     1.036559
5     1.169892
6     1.036559
7     0.969892
8     0.569892
9     0.425806
10    0.303226
11    0.703226
12    0.503226
13    0.492473
14    0.569892
15    0.703226
16    0.425806
17    0.492473
18    0.569892
19    0.479570
20    0.369892
21    0.412903
22    0.569892
23    0.279570
24    0.503226
25    0.397849
26    0.812903
27    0.346237
28    0.197849
29    0.464516
30    0.116129
31    0.062366
dtype: float64

In [119]:
df_heatmap.var(axis=1)

player
Bryce Young           3.236559
Will Anderson Jr      1.303226
CJ Stroud             0.836559
Tyree Wilson          0.636559
Jalen Carter          0.969892
Devon Witherspoon     1.569892
Anthony Richardson    0.703226
Will Levis            0.425806
Christian Gonzalez    0.436559
Peter Skoronski       0.503226
Paris Johnson Jr      0.769892
Lukas Van Ness        0.569892
Broderick Jones       0.769892
Nolan Smith           0.412903
Bijan Robinson        0.636559
Jaxon Smith-Njigba    0.503226
Joey Porter Jr        0.703226
Darnell Wright        0.511828
Myles Murphy          0.292473
Quentin Johnston      0.247312
Jordan Addison        0.492473
Deonte Banks          0.359140
Emmanuel Forbes       0.227957
Brian Branch          0.479570
Dalton Kincaid        0.569892
Jahmyr Gibbs          0.116129
O'Cyrus Torrence      0.206452
Zay Flowers           0.492473
Bryan Bresee          0.380645
Calijah Kancey        0.313978
Michael Mayer         0.625806
dtype: float64